In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="BostonHousing"
JobNameAbbrev = "BH"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "rashomon_bound_adder": [0.015],
                       "Type": ["Classification"]}

# ParameterDictionary = {"Data":[Data],
#                        "Seed":list(range(0,100)),
#                        "TestProportion":[0.2],
#                        "CandidateProportion":[0.8],
#                        "SelectorType":["TreeEnsembleQBCFunction"],
#                        "ModelType":["TreeFarmsFunction"],
#                        "UniqueErrorsInput": [0,1],
#                        "n_estimators": [100], 
#                        "regularization": [0.01],
#                        "rashomon_bound_adder": [0.015],
#                        "Type": ["Classification"]}


# Create Parameter Vector

In [3]:
# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Filter #
IndexDrop = ParameterVector[(ParameterVector["ModelType"] == "RandomForestClassificationFunction") & (ParameterVector["UniqueErrorsInput"]== 1)].index
ParameterVector.drop(IndexDrop , inplace=True)
ParameterVector.index = range(0,ParameterVector.shape[0])

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["rashomon_bound_adder"].astype(str)
    )

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_DPL", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_UNQ", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True)
)


In [4]:
ParameterVector["JobName"][0]

'0BH_RF0.015'

In [5]:
# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

In [6]:
ParameterVector["Output"][0]

'BostonHousing/RandomForestClassification/Raw/0BH_RF0.015.pkl'

# Save Parameter Vector

In [7]:
# Save 
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)

In [8]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,rashomon_bound_adder,Type,JobName,Output
0,BostonHousing,0,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,0BH_RF0.015,BostonHousing/RandomForestClassification/Raw/0...
1,BostonHousing,1,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,1BH_RF0.015,BostonHousing/RandomForestClassification/Raw/1...
2,BostonHousing,2,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,2BH_RF0.015,BostonHousing/RandomForestClassification/Raw/2...
3,BostonHousing,3,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,3BH_RF0.015,BostonHousing/RandomForestClassification/Raw/3...
4,BostonHousing,4,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,4BH_RF0.015,BostonHousing/RandomForestClassification/Raw/4...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,BostonHousing,95,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,95BH_RF0.015,BostonHousing/RandomForestClassification/Raw/9...
96,BostonHousing,96,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,96BH_RF0.015,BostonHousing/RandomForestClassification/Raw/9...
97,BostonHousing,97,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,97BH_RF0.015,BostonHousing/RandomForestClassification/Raw/9...
98,BostonHousing,98,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,0.015,Classification,98BH_RF0.015,BostonHousing/RandomForestClassification/Raw/9...
